# Jointure des fichiers

Le but de ce notebook est de produire un fichier csv utilisable pour la suite de la construction de notre modèle. C'est-à-partir de ce fichier que seront effectué les nettoyages et autres traitements préalables (imputation à la moyenne/médiane, etc.).

In [1]:
# Importation des librairies nécessaires à la manipulation de df et array

import pandas as pd
import numpy as np

In [2]:
# Création de pathfile à modifier en fonction de l'organisation de notre dossier

pathfile_raw = '/Users/cecil/Documents/IFN_Mednum/raw/'
pathfile_interim = '/Users/cecil/Documents/IFN_Mednum/interim/'
pathfile_external = '/Users/cecil/Documents/IFN_Mednum/external/'
pathfile_processed = '/Users/cecil/Documents/IFN_Mednum/processed/'

In [3]:
# Affichage du dataframe

data = pd.read_csv(f'{pathfile_processed}data_merge.csv')
data.head()

,Unnamed: 0,CODE_INSEE,TX_POVERTY,TX_NSCOL15P,TX_MENSEUL,TX_FAMMONO,TX_65ETPLUS,TX_25ETMOINS,TX_DEMANDEUR_EMPLOIS,COUVERTURE_HD_THD
0,0,2A001,NaN,47.724584,22.350501,9.322385,21.662567,10.902031,NaN,1.0
1,1,2A004,27.441333,47.006382,26.648289,14.049949,26.619498,11.425760,13.062663,1.0
2,2,2A006,17.826087,45.656990,20.245461,9.324807,21.224153,10.888846,NaN,1.0
3,3,2A008,NaN,41.227734,33.376123,9.627728,36.385084,6.076146,NaN,1.0
4,4,2A011,NaN,47.727273,48.148148,3.703704,40.909091,11.363636,NaN,1.0


In [4]:
data = data.drop(columns='Unnamed: 0')

In [5]:
df = pd.read_csv(f'{pathfile_external}table_iris_insee_epci.csv')

In [6]:
df = df[['CODE_INSEE', 'LIBCOM', 'DEP']]
df.head()

,CODE_INSEE,LIBCOM,DEP
0,01001,L'Abergement-Clémenciat,01
1,01002,L'Abergement-de-Varey,01
2,01004,Ambérieu-en-Bugey,01
3,01004,Ambérieu-en-Bugey,01
4,01004,Ambérieu-en-Bugey,01


In [7]:
# Jointure pour avoir les informations des communes avec notre dataframe

data = pd.merge(data, df, left_on='CODE_INSEE', right_on='CODE_INSEE')
data = data[['CODE_INSEE', 'DEP', 'LIBCOM', 'TX_POVERTY', 'TX_NSCOL15P', 'TX_MENSEUL', 'TX_FAMMONO', 'TX_65ETPLUS', 'TX_25ETMOINS', 'TX_DEMANDEUR_EMPLOIS', 'COUVERTURE_HD_THD']]
data.head()

,CODE_INSEE,DEP,LIBCOM,TX_POVERTY,TX_NSCOL15P,TX_MENSEUL,TX_FAMMONO,TX_65ETPLUS,TX_25ETMOINS,TX_DEMANDEUR_EMPLOIS,COUVERTURE_HD_THD
0,2A001,2A,Afa,NaN,47.724584,22.350501,9.322385,21.662567,10.902031,NaN,1.0
1,2A004,2A,Ajaccio,27.441333,47.006382,26.648289,14.049949,26.619498,11.425760,13.062663,1.0
2,2A004,2A,Ajaccio,27.441333,47.006382,26.648289,14.049949,26.619498,11.425760,13.062663,1.0
3,2A004,2A,Ajaccio,27.441333,47.006382,26.648289,14.049949,26.619498,11.425760,13.062663,1.0
4,2A004,2A,Ajaccio,27.441333,47.006382,26.648289,14.049949,26.619498,11.425760,13.062663,1.0


In [8]:
# Permet de supprimer les doublons sans perdre de données

data = data.groupby(['CODE_INSEE', 'DEP', 'LIBCOM']).mean().reset_index()

In [9]:
# Ajout d'une colonne "Tx_BENEF_MINIMAS_SOC" en attendant les données

data['TX_BENEF_MINIMAS_SOC'] = 1
data.head()

,CODE_INSEE,DEP,LIBCOM,TX_POVERTY,TX_NSCOL15P,TX_MENSEUL,TX_FAMMONO,TX_65ETPLUS,TX_25ETMOINS,TX_DEMANDEUR_EMPLOIS,COUVERTURE_HD_THD,TX_BENEF_MINIMAS_SOC
0,01001,01,L'Abergement-Clémenciat,NaN,52.154281,22.823614,4.831504,23.779236,11.309305,NaN,1.0,1
1,01002,01,L'Abergement-de-Varey,NaN,40.425532,28.571429,4.761905,23.404255,7.978723,NaN,1.0,1
2,01004,01,Ambérieu-en-Bugey,26.170897,49.474587,41.515231,10.216084,20.733915,16.620013,NaN,1.0,1
3,01005,01,Ambérieux-en-Dombes,NaN,53.574678,21.076206,6.601983,19.488442,13.854125,NaN,1.0,1
4,01006,01,Ambléon,NaN,53.061224,36.363636,9.090909,26.530612,10.204082,NaN,1.0,1


In [10]:
# Affichage des valeurs manquantes et dupliquées

print('Valeurs manquantes :\n' + str(data.isna().sum()))
print('Valeurs dupliquées : ', data.duplicated().sum())

Valeurs manquantes :
CODE_INSEE                  0
DEP                         0
LIBCOM                      0
TX_POVERTY              29428
TX_NSCOL15P                24
TX_MENSEUL                 31
TX_FAMMONO                 31
TX_65ETPLUS                24
TX_25ETMOINS               24
TX_DEMANDEUR_EMPLOIS    34938
COUVERTURE_HD_THD           0
TX_BENEF_MINIMAS_SOC        0
dtype: int64
Valeurs dupliquées :  0


In [11]:
data.to_csv(f'{pathfile_processed}data_merge_V2.csv')